# 캠페인과 상호 작용 <a class="anchor" id="top"></a>

이 노트북에서는 Amazon Personalize에서 캠페인을 배포하고 상호 작용합니다.

1. [소개](#intro)
1. [캠페인 생성](#create)
1. [캠페인과 상호 작용](#interact)
1. [배치 추천](#batch)
1. [마무리](#wrapup)

## 소개 <a class="anchor" id="intro"></a>
[맨 위로 이동](#top)

이 시점에는 몇 개의 솔루션과 각 솔루션의 버전이 하나 만들어져 있습니다. 일단 솔루션 버전이 생성되면 해당 버전으로부터 추천을 받고 전반적인 동작을 파악할 수 있습니다.

이 노트북에서는 이전 노트북의 각 솔루션 버전을 개별 캠페인에 배포하는 것부터 시작합니다. 캠페인이 활성화되면 추천을 쿼리하기 위한 리소스와 사용자에게 좀 더 읽기 쉬운 출력으로 요약하는 헬퍼 함수가 제공됩니다.

Amazon Personalize를 사용하는 고객과 마찬가지로, 헬퍼 함수를 데이터 입력 파일의 구조에 맞게 수정하여 추가 렌더링이 계속 작동하도록 할 수 있습니다.

시작하려면 라이브러리를 가져오고 이전 노트북의 값을 로드하고 SDK를 로드해야 합니다.

In [ ]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random

import boto3
import pandas as pd

In [ ]:
%store -r

In [ ]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## 캠페인과 상호 작용 <a class="anchor" id="interact"></a>
[맨 위로 이동](#top)

이제 모든 캠페인이 배포되고 활성화되었으므로 API 호출을 통해 추천을 받을 수 있습니다. 각 캠페인은 약간 다른 방식으로 작동하는 다른 레시피를 기반으로, 서로 다른 사용 사례를 지원합니다. 각 캠페인은 이전 노트북과 다른 순서로 진행되며, 복잡성이 발생할 수 있는 경우 오름차순(즉, 가장 간단한 것부터)에 따라 처리됩니다.

먼저 Personalize 캠페인이 반환한 결과를 이해할 수 있도록 지원 함수를 만들어 보겠습니다. Personalize는 `item_id`만 반환합니다. 이 함수는 데이터를 압축하기에 좋지만, 사용자가 읽을 수 있는 노트북 결과를 얻으려면 데이터베이스나 조회 테이블을 쿼리해야 합니다. 헬퍼 함수를 생성하여 FM 데이터 세트에서 사용자가 읽을 수 있는 결과를 반환하겠습니다.

먼저 조회 테이블에 사용할 수 있는 데이터 세트를 로드합니다.

In [ ]:
# Create a dataframe for the items by reading in the correct source CSV
items_df = pd.read_csv(dataset_dir + '/movies.csv', sep=',', usecols=[0,1], encoding='latin-1', dtype={'movieId': "object", 'title': "str"},index_col=0)

# Render some sample data
items_df.head(5)

ID 열을 인덱스 열로 정의하면 ID를 쿼리하는 것만으로 아티스트를 반환할 수 있습니다.

In [ ]:
movie_id_example = 589
title = items_df.loc[movie_id_example]['title']
print(title)

그렇게 나쁘지는 않지만, 코드의 모든 곳에서 이 작업을 반복하면 더 복잡해집니다. 그래서 아래의 함수가 이를 정리해줄 것입니다.

In [ ]:
def get_movie_by_id(movie_id, movie_df=items_df):
    """
    This takes in an artist_id from Personalize so it will be a string,
    converts it to an int, and then does a lookup in a default or specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return movie_df.loc[int(movie_id)]['title']
    except:
        return "Error obtaining title"

이제 오류 캐칭 기능을 확인하기 위해 몇 가지 간단한 값을 테스트해 보겠습니다.

In [ ]:
# A known good id (The Princess Bride)
print(get_movie_by_id(movie_id="1197"))
# A bad type of value
print(get_movie_by_id(movie_id="987.9393939"))
# Really bad values
print(get_movie_by_id(movie_id="Steve"))

좋습니다. 이제 결과를 렌더링할 수 있습니다. 

### SIMS

SIMS는 입력으로 항목만 요구하며 입력 항목과 유사한 방식으로 사용자가 상호 작용하는 항목을 반환합니다. 이 예의 경우 그 항목은 바로 영화입니다. 

아래의 셀은 SIMS로부터 추천을 받고 결과를 렌더링하는 작업을 처리합니다. 이 노트북의 앞부분에서 살펴본 첫 번째 항목(터미네이터 2: 심판의 날)에 대한 추천을 살펴보겠습니다.

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = sims_campaign_arn,
    itemId = str(589),
)

In [ ]:
item_list = get_recommendations_response['itemList']
for item in item_list:
    print(get_movie_by_id(movie_id=item['itemId']))

축하합니다! 첫 번째 추천 목록을 받았습니다. 이 목록도 괜찮지만, 아티스트 샘플 컬렉션에 대한 추천을 멋진 데이터 프레임으로 렌더링하는 것이 좋습니다. 다시 한 번 헬퍼 함수를 만들어 보겠습니다.

In [ ]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df(recommendations_df, movie_ID):
    # Get the movie name
    movie_name = get_movie_by_id(movie_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = sims_campaign_arn,
        itemId = str(movie_ID),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

이제 여러 편의 영화로 헬퍼 함수를 테스트해 보겠습니다. SIMS 캠페인을 테스트하기 위해 데이터 세트에서 몇 가지 데이터를 추출해 보겠습니다. 데이터 프레임에서 임의의 영화 5편을 선택합니다.

참고: 유사한 제목을 표시하므로 나열된 일부 영화를 인식할 때까지 샘플을 다시 실행할 수 있습니다.

In [ ]:
samples = items_df.sample(5)
samples

In [ ]:
sims_recommendations_df = pd.DataFrame()
movies = samples.index.tolist()

for movie in movies:
    sims_recommendations_df = get_new_recommendations_df(sims_recommendations_df, movie)

sims_recommendations_df

많은 항목이 동일해 보이지만 모든 항목이 동일하지는 않을 수 있습니다(이 경우 상호 작용 횟수가 적을 가능성이 더 높으며, 이는 movielens 소규모 데이터 세트에서 더 일반적). 이는 솔루션 버전을 평가할 때 평가 지표에만 의존해서는 안 된다는 것을 보여줍니다. 그러면 이런 일이 일어날 때, 어떻게 하면 결과를 개선할 수 있을까요?

이쯤에서 개인화 레시피의 하이퍼파라미터에 대해 생각해 보는 것이 좋습니다. SIMS 레시피에는 `popularity_discount_factor` 하이퍼파라미터가 있습니다([설명서](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-sims.html) 참조). 이 하이퍼파라미터를 활용하면 결과에 나타나는 뉘앙스를 제어할 수 있습니다. 이 파라미터와 그 동작은 사용자가 접하는 모든 데이터 세트에 고유하며 비즈니스 목표에 따라 달라집니다. 결과에 만족할 때까지 이 하이퍼파라미터 값을 반복하거나, Personalize의 하이퍼파라미터 최적화(HPO) 기능을 활용하여 시작할 수 있습니다. 하이퍼파라미터 및 HPO 튜닝에 대한 자세한 내용은 [설명서](https://docs.aws.amazon.com/personalize/latest/dg/customizing-solution-config-hpo.html)를 참조하세요.

### 사용자 개인화

HRNN은 Amazon Personalize에서 제공하는 고급 알고리즘 중 하나입니다. 이 알고리즘은 특정 사용자의 과거 동작을 기준으로 항목을 개인화할 수 있도록 지원하며, 다시 훈련하지 않고도 사용자에 대한 추천을 변경하기 위해 실시간 이벤트를 가져올 수 있습니다.

HRNN은 사용자의 샘플을 사용하므로, 샘플로 사용할 데이터를 로드하고 임의의 사용자 3명을 선택합니다. Movielens는 사용자 데이터를 포함하지 않기 때문에 데이터 세트의 사용자 ID 범위에서 3개의 난수를 선택합니다.

In [ ]:
if not USE_FULL_MOVIELENS:
    users = random.sample(range(1, 600), 3)
else:
    users = random.sample(range(1, 162000), 3)
users

이제 위에서 선택한 임의의 사용자 3명을 위한 추천을 렌더링하겠습니다. 그런 다음 개인별 순위로 넘어가기 전에 실시간 상호 작용을 살펴보도록 하겠습니다.

다시 말하지만, 결과를 멋진 데이터 프레임으로 렌더링하는 헬퍼 함수를 만듭니다.

#### API 호출 결과

In [ ]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_users(recommendations_df, user_id):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    #print(recommendation_list)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [user_id])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [ ]:
recommendations_df_users = pd.DataFrame()
#users = users_df.sample(3).index.tolist()

for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

여기에서, 각 사용자마다 추천이 서로 다르다는 것을 분명히 알 수 있습니다. 이러한 결과에 캐시가 필요한 경우 먼저 모든 사용자에 대해 API 호출을 실행하고 결과를 저장하거나, 이 노트북의 뒷부분에서 다룰 배치 내보내기를 사용할 수 있습니다.

이제 항목 필터를 적용하여 장르 내에서 이러한 사용자 중 하나에 대한 추천을 확인하겠습니다.


In [ ]:
def get_new_recommendations_df_by_filter(recommendations_df, user_id, filter_arn):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
        filterArn = filter_arn
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    #print(recommendation_list)
    filter_name = filter_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

특정 장르 내에서 추천하는 영화를 확인할 수 있습니다. VOD 애플리케이션 내에서 이러한 필터를 사용하여 선반(레일 또는 캐로셀이라고도 함)을 쉽게 만들 수 있습니다. 항목에 대한 정보에 따라 키워드, 연도/연대 등의 추가 정보를 필터링할 수도 있습니다.

In [ ]:
recommendations_df_shelves = pd.DataFrame()
for filter_arn in meta_filter_arns:
    recommendations_df_shelves = get_new_recommendations_df_by_filter(recommendations_df_shelves, user, filter_arn)
for filter_arn in decade_filter_arns:
    recommendations_df_shelves = get_new_recommendations_df_by_filter(recommendations_df_shelves, user, filter_arn)

recommendations_df_shelves

다음 주제는 실시간 이벤트입니다. Personalize에는 사용자에게 표시되는 추천을 업데이트하기 위해 프로그램에서 이벤트를 수신하는 기능이 있습니다. 이 기능은 온디맨드 비디오와 같은 미디어 워크로드에서 특히 유용합니다. 온디맨드 비디오의 경우 자녀와 함께 시청하는지 아니면 혼자 시청하는지에 따라 고객의 의도가 달라질 수 있습니다.

또한 이 시스템을 통해 기록된 이벤트는 삭제 호출이 실행될 때까지 저장되며, 다음 모델을 훈련할 때 제공하는 다른 상호 작용 데이터와 함께 기록 데이터로 사용됩니다.

#### 실시간 이벤트

캠페인에 연결되는 이벤트 트래커를 만드는 것부터 시작합니다.

In [ ]:
response = personalize.create_event_tracker(
    name='MovieTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']
event_tracker_arn = response['eventTrackerArn']

특정 항목과 상호 작용하는 사용자를 시뮬레이션하는 코드를 만듭니다. 이 코드를 실행한 후에는 위의 결과와 다른 추천을 받게 됩니다.

먼저 실시간 이벤트 시뮬레이션을 위한 몇 가지 메서드를 생성합니다.

In [ ]:
session_dict = {}

def send_movie_click(USER_ID, ITEM_ID, EVENT_TYPE):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[str(USER_ID)]
    except:
        session_dict[str(USER_ID)] = str(uuid.uuid1())
        session_ID = session_dict[str(USER_ID)]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= str(USER_ID),
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': str(EVENT_TYPE),
        'properties': event_json
        }]
    )

def get_new_recommendations_df_users_real_time(recommendations_df, user_id, item_id, event_type):
    # Get the artist name (header of column)
    movie_name = get_movie_by_id(item_id)
    # Interact with different movies
    print('sending event ' + event_type + ' for ' + get_movie_by_id(item_id))
    send_movie_click(USER_ID=user_id, ITEM_ID=item_id, EVENT_TYPE=event_type)
    # Get the recommendations (note you should have a base recommendation DF created before)
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        artist = get_movie_by_id(item['itemId'])
        recommendation_list.append(artist)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    #recommendations_df = recommendations_df.join(new_rec_DF)
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

이 시점에는 아직 실시간 이벤트를 생성하지 않았으며 코드만 설정했습니다. 실시간 이벤트 전후의 추천을 비교하기 위해, 한 명의 사용자를 선택하고 해당 사용자에 대한 원래 추천을 생성해 보겠습니다.

In [ ]:
# First pick a user
user_id = user

# Get recommendations for the user
get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    artist = get_movie_by_id(item['itemId'])
    recommendation_list.append(artist)
user_recommendations_df = pd.DataFrame(recommendation_list, columns = [user_id])
user_recommendations_df

실시간 이벤트를 적용하기 전의 이 사용자에 대한 추천 목록이 있습니다. 이제 사용자의 상호 작용을 시뮬레이션할 임의의 아티스트 3명을 선택하고 그에 따라 추천이 어떻게 바뀌는지 살펴보겠습니다.

In [ ]:
# Next generate 3 random movies
movies = items_df.sample(3).index.tolist()

In [ ]:
# Note this will take about 15 seconds to complete due to the sleeps
for movie in movies:
    user_recommendations_df = get_new_recommendations_df_users_real_time(user_recommendations_df, user_id, movie,'click')
    time.sleep(5)

이제 클릭 이벤트로 인해 추천이 어떻게 변경되었는지 살펴볼 수 있습니다.

In [ ]:
user_recommendations_df

위의 셀에서 인덱스 뒤의 첫 번째 열은 사용자 개인화에서 사용자의 기본 추천이며, 그 이후의 각 열에는 실시간 이벤트를 통해 상호 작용한 아티스트의 헤더와 이 이벤트가 발생한 후의 추천이 있습니다.

이러한 동작은 크게 바뀌지 않을 수 있습니다. 이는 이 데이터 세트의 특성과 몇 번의 무작위 클릭의 효과로 인해 상대적으로 제한되기 때문입니다. 이를 더 잘 이해하기 위해 더 많은 영화를 클릭하는 경우를 시뮬레이션을 해보면, 더 뚜렷한 효과를 볼 수 있습니다.

이제 상호 작용 데이터를 기준으로 항목을 필터링할 수 있는 이벤트 필터를 살펴보겠습니다. 이 데이터 세트의 경우 가져온 데이터를 기반으로 클릭하거나 시청할 수 있지만, 사용자가 설계하는 모든 상호 작용 스키마(클릭, 속도, 좋아요, 시청, 구매 등)에 기반할 수 있습니다. VOD 쉘프의 경우 타이틀을 "Top picks for you (고객님을 위한 탑 픽)"에서 "Watch again"로 옮길 수 있습니다. 다시 보기 추천은 사용자의 현재 상호 작용을 기반으로 하지만 이미 시청한 타이틀만 추천합니다.


In [ ]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_recommendations_df_by_filter(recommendations_df_events, user, filter_arn)
    
recommendations_df_events

이제 시청하지 않은 4개의 추천에 대해 4개의 영화를 시청하는 경우를 시뮬레이션하는 시청 이벤트를 보내겠습니다. VOD 애플리케이션에서 콘텐츠의 상당량(75%)을 시청하고 나면 이벤트를 보내도록 선택할 수 있습니다. 100% 완료 시를 기준으로 이벤트를 보내면 크레딧을 마저 보지 않은 사람을 놓칠 수 있습니다.

In [ ]:
 # Get the recommendations
top_unwatched_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = userpersonalization_campaign_arn,
    userId = str(user_id),
    filterArn = filter_arn,
    numResults=4)
item_list = top_unwatched_recommendations_response['itemList']
for item in item_list:
    print('sending event watch for ' + get_movie_by_id(item['itemId']))
    send_movie_click(USER_ID=user_id, ITEM_ID=item['itemId'], EVENT_TYPE='watch')
    time.sleep(10)

이제 이벤트 필터를 확인하여 업데이트된 시청한 추천과 시청하지 않은 추천을 확인할 수 있습니다. 

In [ ]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_recommendations_df_by_filter(recommendations_df_events, user, filter_arn)
    
recommendations_df_events

### 개인별 순위

개인화된 순위의 핵심 사용 사례는 항목 컬렉션을 가지고, 사용자에게 우선 순위 또는 가능한 관심도 순서로 렌더링하는 것입니다. VOD 애플리케이션의 경우 일부 정보(감독, 로케이션, 슈퍼히어로 프랜차이즈, 영화의 시대 배경 등)에 따라 개인화된 쉘프/레일/캐러셀을 동적으로 렌더링합니다. 이는 메타데이터에 있는 정보가 아니므로 항목 메타데이터 필터가 작동하지 않을 수 있지만, 이 정보를 사용하여 항목 목록을 생성할 수 있습니다.

이를 보여주기 위해 이전과 동일한 사용자와 임의의 항목 컬렉션을 사용합니다.

In [ ]:
rerank_user = user
rerank_items = items_df.sample(25).index.tolist()

이제 입력 데이터를 보여주는 멋진 데이터 프레임을 구축합니다.

In [ ]:
rerank_list = []
for item in rerank_items:
    movie = get_movie_by_id(item)
    rerank_list.append(movie)
rerank_df = pd.DataFrame(rerank_list, columns = ['Un-Ranked'])
rerank_df

그런 다음 개인화된 순위 API를 호출합니다.

In [ ]:
# Convert user to string:
user_id = str(rerank_user)
rerank_item_list = []
for item in rerank_items:
    rerank_item_list.append(str(item))
    
# Get recommended reranking
get_recommendations_response_rerank = personalize_runtime.get_personalized_ranking(
        campaignArn = rerank_campaign_arn,
        userId = user_id,
        inputList = rerank_item_list
)

나란히 비교할 수 있도록, 순위가 다시 매겨진 항목을 원래 데이터 프레임에 두 번째 열로 추가합니다.

In [ ]:
ranked_list = []
item_list = get_recommendations_response_rerank['personalizedRanking']
for item in item_list:
    movie = get_movie_by_id(item['itemId'])
    ranked_list.append(movie)
ranked_df = pd.DataFrame(ranked_list, columns = ['Re-Ranked'])
rerank_df = pd.concat([rerank_df, ranked_df], axis=1)
rerank_df

위에서 모델의 사용자에 대한 이해를 바탕으로 각 항목이 어떻게 다시 정렬되었는지 확인할 수 있습니다. 이 작업은 사용자에게 노출할 항목의 컬렉션(예: 프로모션 목록)이 있을 때 흔히 사용됩니다.

## 배치 추천 <a class="anchor" id="batch"></a>
[맨 위로 이동](#top)

내보낸 추천으로 이루어진 더 큰 데이터 세트가 필요한 경우가 많습니다. 최근 Amazon Personalize는 추천 컬렉션을 S3로 내보내는 수단으로서 배치 추천을 발표했습니다. 이 예에서는 HRNN 솔루션에 대해 이 작업을 수행하는 방법을 살펴보겠습니다. 배치 추천에 대한 자세한 내용은 [설명서](https://docs.aws.amazon.com/personalize/latest/dg/getting-recommendations.html#recommendations-batch)를 참조하세요. 이 기능은 모든 레시피에 적용되지만 출력 형식은 레시피마다 다릅니다.

간단한 구현의 예는 다음과 같습니다.

```python
import boto3

personalize_rec = boto3.client(service_name='personalize')

personalize_rec.create_batch_inference_job (
    solutionVersionArn = "Solution version ARN",
    jobName = "Batch job name",
    roleArn = "IAM role ARN",
    jobInput = 
       {"s3DataSource": {"path": S3 input path}},
    jobOutput = 
       {"s3DataDestination": {"path":S3 output path"}}
)
```

SDK 가져오기, 솔루션 버전 알림 및 역할 알림이 모두 결정되었습니다. 이제 입력, 출력 및 작업 이름만 정의하면 됩니다.

HRNN에 대한 입력부터 시작하겠습니다. 이 입력은 다음과 같습니다.


```JSON
{"userId": "4638"}
{"userId": "663"}
{"userId": "3384"}
```

이 경우 다음과 같은 출력이 생성됩니다.

```JSON
{"input":{"userId":"4638"}, "output": {"recommendedItems": ["296", "1", "260", "318"]}}
{"input":{"userId":"663"}, "output": {"recommendedItems": ["1393", "3793", "2701", "3826"]}}
{"input":{"userId":"3384"}, "output": {"recommendedItems": ["8368", "5989", "40815", "48780"]}}
```

출력은 JSON Lines 파일로, 한 줄에 하나씩 개별 JSON 객체로 구성됩니다. 따라서 나중에 결과를 이 형식으로 수집하기 위해서는 추가 작업이 필요합니다.

### 입력 파일 생성

배치 기능을 사용할 경우, 작업이 완료되었을 때 추천을 받을 사용자를 지정합니다. 아래의 셀은 다시 몇 명의 임의의 사용자를 선택한 후 파일을 만들고 디스크에 저장합니다. 그런 다음 S3에 업로드하여 나중에 API 호출에 사용할 수 있습니다.

In [ ]:
# We will use the same users from before
users
# Write the file to disk
json_input_filename = "json_input.json"
with open(data_dir + "/" + json_input_filename, 'w') as json_input:
    for user_id in users:
        json_input.write('{"userId": "' + str(user_id) + '"}\n')

In [ ]:
# Showcase the input file:
!cat $data_dir"/"$json_input_filename

파일을 S3에 업로드하고 나중에 사용할 수 있도록 경로를 변수로 저장합니다.

In [ ]:
# Upload files to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(json_input_filename).upload_file(data_dir+"/"+json_input_filename)
s3_input_path = "s3://" + bucket_name + "/" + json_input_filename
print(s3_input_path)

배치 추천은 S3에 업로드한 파일에서 입력 데이터를 읽습니다. 마찬가지로 배치 추천은 출력을 S3의 파일에 저장합니다. 따라서 결과를 저장할 출력 경로를 정의합니다.

In [ ]:
# Define the output path
s3_output_path = "s3://" + bucket_name + "/"
print(s3_output_path)

이제 배치 내보내기 프로세스를 시작하는 호출을 실행하면 됩니다.

In [ ]:
batchInferenceJobArn = personalize.create_batch_inference_job (
    solutionVersionArn = userpersonalization_solution_version_arn,
    jobName = "VOD-POC-Batch-Inference-Job-UserPersonalization_" + str(round(time.time()*1000)),
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

배치 추천 호출의 상태를 추적하기 위해 아래의 while 루프를 실행합니다. Personalize는 이 작업을 수행하기 위해 인프라를 구축해야 하므로, 작업을 완료하는 데 30분 정도 걸릴 수 있습니다. 여기서는 3명의 사용자로 이루어진 데이터 세트로 이 기능을 테스트하고 있는데, 이는 이 메커니즘의 효율적인 사용 사례라고 할 수 없습니다. 일반적으로 이 기능은 대량 처리에만 사용되며, 이 경우 높은 효율성이 명확하게 나타납니다.

In [ ]:
current_time = datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

In [ ]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket_name, export_name, data_dir+"/"+export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open(data_dir+"/"+export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the artists
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            movie = get_movie_by_id(item)
            recommendation_list.append(movie)
        if 'bulk_recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            bulk_recommendations_df = bulk_recommendations_df.join(new_rec_DF)
        else:
            bulk_recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])
        try:
            line = json.loads(json_file.readline())
        except:
            line = None
bulk_recommendations_df

## 마무리 <a class="anchor" id="wrapup"></a>
[맨 위로 이동](#top)

이를 통해 다양한 추천 및 개인화 시나리오를 처리할 수 있는 완전한 모델 컬렉션과 고객 데이터를 조작하여 서비스와 보다 효과적으로 통합할 수 있는 기술, API를 통해 오픈 소스 데이터 과학 도구를 활용하여 이 모든 것을 수행하는 방법에 대한 지식을 갖출 수 있습니다.

이러한 노트북을 가이드로 사용하여 고객과 POC를 시작할 수 있습니다. 누락된 구성 요소를 찾거나 새 접근 방식을 발견한 경우, 풀 요청을 잘라내고 이 컬렉션에서 누락된 유용한 구성 요소를 추가로 제공합니다.

이 POC를 진행하는 동안 배포한 모든 리소스를 정리해야 합니다. `06_Clean_Up_Resources.ipynb`의 리소스를 식별하고 삭제하는 방법을 보여 주는 별도의 노트북을 제공했습니다.

In [ ]:
%store event_tracker_arn
%store batchInferenceJobArn